# Stock Basics: The Building Blocks of Investing

**Time:** 15 minutes

**Prerequisites:** None - this is where we start!

---

Before we build AI agents that analyze stocks, we need to understand what stocks are and the key data points that define them. This notebook covers the fundamentals that every investor (and AI agent) needs to know.

## What is a Stock?

A **stock** (also called a share or equity) represents **ownership in a company**.

When you buy a stock, you become a **shareholder** - you own a tiny piece of that company.

**Mental Model: The Pizza Analogy**

Imagine a company is a pizza:
- The whole pizza = the entire company
- Each slice = one share of stock
- The more slices you own, the bigger your ownership stake

If the pizza (company) gets more valuable, your slices become worth more too!

## Ticker Symbols: The Stock's Nickname

Every publicly traded company has a **ticker symbol** - a short code that identifies it.

| Company | Ticker | Why This Symbol? |
|---------|--------|------------------|
| Apple Inc. | AAPL | Abbreviated name |
| Microsoft | MSFT | Abbreviated name |
| Tesla | TSLA | Abbreviated name |
| Alphabet (Google) | GOOGL | Original name |
| Amazon | AMZN | Abbreviated name |
| NVIDIA | NVDA | Abbreviated name |

**Key Insight:** Tickers are what we use in code and APIs. When your AI agent needs Apple's stock price, it asks for `"AAPL"`, not `"Apple Inc."`

In [ ]:
# Install required package
!pip install -q yfinance pandas

In [ ]:
import yfinance as yf

# Using ticker symbols to get stock data
# The ticker is how we "address" a company in code

apple = yf.Ticker("AAPL")  # Create a Ticker object for Apple
nvidia = yf.Ticker("NVDA")  # Create a Ticker object for NVIDIA

print(f"Apple's ticker: AAPL")
print(f"NVIDIA's ticker: NVDA")
print("\nWe'll use these objects to fetch real data!")

Apple's ticker: AAPL
NVIDIA's ticker: NVDA

We'll use these objects to fetch real data!


In [ ]:
# Get current stock prices
# Note: 'currentPrice' or 'regularMarketPrice' depending on market hours

def get_current_price(ticker_symbol):
    """Get the current price for a stock."""
    ticker = yf.Ticker(ticker_symbol)
    info = ticker.info

    # Try different price fields (depends on market hours)
    price = info.get('currentPrice') or info.get('regularMarketPrice') or info.get('previousClose')
    return price

# Get prices for several stocks
stocks = ["AAPL", "NVDA", "MSFT", "GOOGL"]

print("Current Stock Prices:")
print("-" * 30)
for symbol in stocks:
    price = get_current_price(symbol)
    print(f"{symbol}: ${price:,.2f}")

Current Stock Prices:
------------------------------
AAPL: $249.78
NVDA: $184.62
MSFT: $446.64
GOOGL: $331.13


## Key Price Points: Open, High, Low, Close (OHLC)

Stock prices change throughout the trading day. We track four key price points:

| Term | Definition | Why It Matters |
|------|------------|----------------|
| **Open** | First trade price of the day | Shows overnight sentiment |
| **High** | Highest price during the day | Shows peak demand |
| **Low** | Lowest price during the day | Shows selling pressure |
| **Close** | Last trade price of the day | Most important - used for analysis |

**OHLC** is the foundation of technical analysis and charting.

In [ ]:
# Get OHLC data for Apple
apple = yf.Ticker("AAPL")
info = apple.info

print("Apple (AAPL) - Today's Price Data:")
print("-" * 40)
print(f"Open:  ${info.get('regularMarketOpen', 'N/A'):>10,.2f}")
print(f"High:  ${info.get('regularMarketDayHigh', 'N/A'):>10,.2f}")
print(f"Low:   ${info.get('regularMarketDayLow', 'N/A'):>10,.2f}")
print(f"Close: ${info.get('regularMarketPrice', info.get('previousClose', 'N/A')):>10,.2f}")

# Calculate the day's range
high = info.get('regularMarketDayHigh', 0)
low = info.get('regularMarketDayLow', 0)
if high and low:
    day_range = high - low
    print(f"\nDay's Range: ${day_range:.2f} ({(day_range/low)*100:.2f}% swing)")

Apple (AAPL) - Today's Price Data:
----------------------------------------
Open:  $    249.20
High:  $    250.78
Low:   $    248.38
Close: $    249.78

Day's Range: $2.39 (0.96% swing)


## 52-Week High and Low: The Year's Range

The **52-week high** and **52-week low** show the highest and lowest prices over the past year (52 weeks).

**Why It Matters:**

1. **Technical Analysis:** Traders watch when stocks approach their 52-week high (potential resistance) or low (potential support)

2. **Sentiment Indicator:**
   - Near 52-week high → Bullish momentum, positive sentiment
   - Near 52-week low → Bearish momentum, may be a value opportunity or a warning sign

**The Calculation:** Distance from 52-week high tells you how far the stock has fallen from its peak.

In [ ]:
def analyze_52_week_range(ticker_symbol):
    """Analyze a stock's position within its 52-week range."""
    ticker = yf.Ticker(ticker_symbol)
    info = ticker.info

    current = info.get('currentPrice') or info.get('regularMarketPrice') or info.get('previousClose')
    high_52 = info.get('fiftyTwoWeekHigh')
    low_52 = info.get('fiftyTwoWeekLow')

    if not all([current, high_52, low_52]):
        return None

    # Calculate percentages
    from_high = ((high_52 - current) / high_52) * 100  # % below 52-week high
    from_low = ((current - low_52) / low_52) * 100     # % above 52-week low

    # Position in range (0% = at low, 100% = at high)
    range_position = ((current - low_52) / (high_52 - low_52)) * 100

    return {
        'ticker': ticker_symbol,
        'current': current,
        '52_week_high': high_52,
        '52_week_low': low_52,
        'pct_from_high': from_high,
        'pct_from_low': from_low,
        'range_position': range_position
    }

# Analyze several stocks
stocks = ["AAPL", "NVDA", "MSFT", "TSLA"]

print("52-Week Range Analysis")
print("=" * 70)

for symbol in stocks:
    analysis = analyze_52_week_range(symbol)
    if analysis:
        print(f"\n{analysis['ticker']}:")
        print(f"  Current Price:  ${analysis['current']:,.2f}")
        print(f"  52-Week High:   ${analysis['52_week_high']:,.2f}")
        print(f"  52-Week Low:    ${analysis['52_week_low']:,.2f}")
        print(f"  From High:      {analysis['pct_from_high']:.1f}% below")
        print(f"  From Low:       {analysis['pct_from_low']:.1f}% above")
        print(f"  Range Position: {analysis['range_position']:.1f}% (0%=low, 100%=high)")

52-Week Range Analysis

AAPL:
  Current Price:  $249.77
  52-Week High:   $288.62
  52-Week Low:    $169.21
  From High:      13.5% below
  From Low:       47.6% above
  Range Position: 67.5% (0%=low, 100%=high)

NVDA:
  Current Price:  $184.72
  52-Week High:   $212.19
  52-Week Low:    $86.62
  From High:      12.9% below
  From Low:       113.3% above
  Range Position: 78.1% (0%=low, 100%=high)

MSFT:
  Current Price:  $446.78
  52-Week High:   $555.45
  52-Week Low:    $344.79
  From High:      19.6% below
  From Low:       29.6% above
  Range Position: 48.4% (0%=low, 100%=high)

TSLA:
  Current Price:  $434.75
  52-Week High:   $498.83
  52-Week Low:    $214.25
  From High:      12.8% below
  From Low:       102.9% above
  Range Position: 77.5% (0%=low, 100%=high)


## Market Capitalization: How Big Is the Company?

**Market Cap** = Stock Price × Number of Shares Outstanding

This tells you the **total value of the company** according to the stock market.


In [ ]:
def get_market_cap_analysis(ticker_symbol):
    """Get market cap and classify the company by size."""
    ticker = yf.Ticker(ticker_symbol)
    info = ticker.info

    market_cap = info.get('marketCap', 0)

    # Classify by size
    if market_cap >= 200_000_000_000:  # $200B+
        category = "Mega-cap"
    elif market_cap >= 10_000_000_000:  # $10B+
        category = "Large-cap"
    elif market_cap >= 2_000_000_000:  # $2B+
        category = "Mid-cap"
    elif market_cap >= 300_000_000:  # $300M+
        category = "Small-cap"
    else:
        category = "Micro-cap"

    return {
        'ticker': ticker_symbol,
        'name': info.get('shortName', 'N/A'),
        'market_cap': market_cap,
        'category': category,
        'shares_outstanding': info.get('sharesOutstanding', 0)
    }

# Analyze market caps
stocks = ["AAPL", "NVDA", "MSFT", "GOOGL", "TSLA"]

print("Market Capitalization Analysis")
print("=" * 60)

for symbol in stocks:
    data = get_market_cap_analysis(symbol)
    mc = data['market_cap']

    # Format market cap nicely
    if mc >= 1_000_000_000_000:  # Trillions
        mc_str = f"${mc/1_000_000_000_000:.2f}T"
    else:  # Billions
        mc_str = f"${mc/1_000_000_000:.1f}B"

    print(f"\n{data['ticker']} ({data['name']})")
    print(f"  Market Cap: {mc_str}")
    print(f"  Category:   {data['category']}")

Market Capitalization Analysis

AAPL (Apple Inc.)
  Market Cap: $3.69T
  Category:   Mega-cap

NVDA (NVIDIA Corporation)
  Market Cap: $4.50T
  Category:   Mega-cap

MSFT (Microsoft Corporation)
  Market Cap: $3.32T
  Category:   Mega-cap

GOOGL (Alphabet Inc.)
  Market Cap: $4.01T
  Category:   Mega-cap

TSLA (Tesla, Inc.)
  Market Cap: $1.45T
  Category:   Mega-cap


## Volume: How Many Shares Are Trading?

**Volume** = Number of shares traded in a given period

**Why Volume Matters:**

1. **Liquidity:** High volume means you can buy/sell easily without moving the price
2. **Confirmation:** Price moves with high volume are more meaningful
3. **Interest:** Unusual volume often signals something is happening (news, events)

**Rule of Thumb:**
- Volume > Average Volume → Increased interest
- Volume < Average Volume → Quiet day

In [ ]:
def analyze_volume(ticker_symbol):
    """Analyze trading volume for a stock."""
    ticker = yf.Ticker(ticker_symbol)
    info = ticker.info

    volume = info.get('regularMarketVolume', 0)
    avg_volume = info.get('averageVolume', 0)
    avg_volume_10d = info.get('averageVolume10days', avg_volume)

    # Volume ratio
    volume_ratio = volume / avg_volume if avg_volume > 0 else 0

    return {
        'ticker': ticker_symbol,
        'today_volume': volume,
        'avg_volume': avg_volume,
        'volume_ratio': volume_ratio
    }

# Analyze volume
stocks = ["AAPL", "NVDA", "MSFT", "TSLA"]

print("Volume Analysis")
print("=" * 50)

for symbol in stocks:
    data = analyze_volume(symbol)

    # Interpret the volume
    if data['volume_ratio'] > 1.5:
        interpretation = "HIGH - Unusual interest!"
    elif data['volume_ratio'] > 1.0:
        interpretation = "Above average"
    elif data['volume_ratio'] > 0.7:
        interpretation = "Normal"
    else:
        interpretation = "LOW - Quiet day"

    print(f"\n{data['ticker']}:")
    print(f"  Today's Volume: {data['today_volume']:>15,}")
    print(f"  Avg Volume:     {data['avg_volume']:>15,}")
    print(f"  Volume Ratio:   {data['volume_ratio']:.2f}x average")
    print(f"  Interpretation: {interpretation}")

Volume Analysis

AAPL:
  Today's Volume:      10,131,772
  Avg Volume:          46,204,940
  Volume Ratio:   0.22x average
  Interpretation: LOW - Quiet day

NVDA:
  Today's Volume:      42,345,047
  Avg Volume:         186,760,437
  Volume Ratio:   0.23x average
  Interpretation: LOW - Quiet day

MSFT:
  Today's Volume:       6,627,127
  Avg Volume:          24,641,970
  Volume Ratio:   0.27x average
  Interpretation: LOW - Quiet day

TSLA:
  Today's Volume:      14,560,334
  Avg Volume:          77,633,404
  Volume Ratio:   0.19x average
  Interpretation: LOW - Quiet day


## Putting It All Together: Stock Dashboard

Now let's create a complete stock dashboard that shows all these metrics together.

In [ ]:
def stock_dashboard(ticker_symbol):
    """Create a comprehensive stock dashboard."""
    ticker = yf.Ticker(ticker_symbol)
    info = ticker.info

    # Basic info
    name = info.get('shortName', 'N/A')
    sector = info.get('sector', 'N/A')
    industry = info.get('industry', 'N/A')

    # Price data
    current = info.get('currentPrice') or info.get('regularMarketPrice') or info.get('previousClose')
    open_price = info.get('regularMarketOpen', 0)
    high = info.get('regularMarketDayHigh', 0)
    low = info.get('regularMarketDayLow', 0)
    prev_close = info.get('previousClose', 0)

    # 52-week data
    high_52 = info.get('fiftyTwoWeekHigh', 0)
    low_52 = info.get('fiftyTwoWeekLow', 0)

    # Market cap
    market_cap = info.get('marketCap', 0)

    # Volume
    volume = info.get('regularMarketVolume', 0)
    avg_volume = info.get('averageVolume', 0)

    # Calculate derived metrics
    daily_change = current - prev_close if current and prev_close else 0
    daily_change_pct = (daily_change / prev_close * 100) if prev_close else 0
    from_52_high = ((high_52 - current) / high_52 * 100) if high_52 and current else 0

    # Format market cap
    if market_cap >= 1_000_000_000_000:
        mc_str = f"${market_cap/1_000_000_000_000:.2f}T"
    elif market_cap >= 1_000_000_000:
        mc_str = f"${market_cap/1_000_000_000:.1f}B"
    else:
        mc_str = f"${market_cap/1_000_000:.1f}M"

    # Print dashboard
    print("=" * 60)
    print(f"  {ticker_symbol} - {name}")
    print("=" * 60)
    print(f"  Sector: {sector} | Industry: {industry}")
    print("-" * 60)

    print(f"\n  PRICE")
    change_symbol = "+" if daily_change >= 0 else ""
    print(f"  Current:      ${current:,.2f}  ({change_symbol}{daily_change:.2f}, {change_symbol}{daily_change_pct:.2f}%)")
    print(f"  Open:         ${open_price:,.2f}")
    print(f"  Day Range:    ${low:,.2f} - ${high:,.2f}")

    print(f"\n  52-WEEK RANGE")
    print(f"  Range:        ${low_52:,.2f} - ${high_52:,.2f}")
    print(f"  From High:    {from_52_high:.1f}% below 52-week high")

    print(f"\n  SIZE & LIQUIDITY")
    print(f"  Market Cap:   {mc_str}")
    print(f"  Volume:       {volume:,}")
    print(f"  Avg Volume:   {avg_volume:,}")
    print("=" * 60)

# Create dashboards for several stocks
for ticker in ["AAPL", "NVDA"]:
    stock_dashboard(ticker)
    print()

  AAPL - Apple Inc.
  Sector: Technology | Industry: Consumer Electronics
------------------------------------------------------------

  PRICE
  Current:      $249.85  (+2.20, +0.89%)
  Open:         $249.20
  Day Range:    $248.38 - $250.78

  52-WEEK RANGE
  Range:        $169.21 - $288.62
  From High:    13.4% below 52-week high

  SIZE & LIQUIDITY
  Market Cap:   $3.69T
  Volume:       10,136,332
  Avg Volume:   46,204,940

  NVDA - NVIDIA Corporation
  Sector: Technology | Industry: Semiconductors
------------------------------------------------------------

  PRICE
  Current:      $184.87  (+1.69, +0.92%)
  Open:         $184.75
  Day Range:    $183.93 - $186.16

  52-WEEK RANGE
  Range:        $86.62 - $212.19
  From High:    12.9% below 52-week high

  SIZE & LIQUIDITY
  Market Cap:   $4.50T
  Volume:       42,396,621
  Avg Volume:   186,760,437



## Key Takeaways

### Concepts You've Learned:

1. **Ticker Symbols** - The unique code that identifies each stock (AAPL, NVDA, etc.)

2. **Stock Price** - The current market price per share (but doesn't tell the whole story!)

3. **OHLC** - Open, High, Low, Close - the key price points for each trading day

4. **52-Week Range** - The high and low over the past year, shows where current price sits

5. **Market Cap** - Total company value = price × shares outstanding

6. **Volume** - Number of shares traded, indicates liquidity and interest

### Why This Matters for AI Agents:

When we build AI agents for finance, they'll need to:
- Fetch these data points using APIs
- Calculate derived metrics (like % from 52-week high)
- Interpret what the numbers mean
- Make decisions based on this data

**Next Notebook:** We'll learn about position values - calculating how much your holdings are worth!